# <span style="color:red">OUTDATED: use "Profile NeRF.ipynb" notebook<span>

## Convert NeRF to Timeloop Layer Shapes

In [1]:
import pytorch2timeloop
import os

import sys
sys.path.append("../") # go to parent dir

from accelerating_nerfs.models import VanillaNeRF

In [2]:
# Using vanilla NeRF which are MLPs
model = VanillaNeRF()

# Need to patch the forward method for the purpose of mapping to pass in ray directions
# This ensures the bottleneck layer is captured in the timeloop outputs
model.old_forward = model.forward

def new_forward(self, x):
    return self.old_forward(x, x)

model.forward = new_forward.__get__(model)

input_shape = (1, 3)
batch_size = 2 ** 14
save_dir = "../architectures/layer_shapes"
os.makedirs(save_dir, exist_ok=True)

print(f"input_shape = {input_shape}, batch_size = {batch_size}")

input_shape = (1, 3), batch_size = 16384


In [3]:
pytorch2timeloop.convert_model(
    model=model,
    input_size=input_shape,
    batch_size=batch_size,
    model_name="Nerf",
    save_dir=save_dir,
    convert_fc=True,
    exception_module_names=[],
)
nerf_layer_dir = os.path.join(save_dir, "Nerf")

print(f"Success, check results in {nerf_layer_dir}")

unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'accelerating_nerfs.models.SinusoidalEncoder'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.DenseLayer'>
unknown module type <class 'torch.nn.modules.linear.Identity'>
unknown module type <class 'accelerating_nerfs.models.MLP'>
unknown module type <class 'accelerating_nerfs.models.NerfMLP'>
unknown module type <class 'accelerating_nerfs.models.VanillaNeRF'>


Success, check results in ../architectures/layer_shapes/Nerf


### Print out layer info and check against actual model

In [4]:
import re
import yaml


def natural_sort(l): 
    convert = lambda text: int(text) if text.isdigit() else text.lower()
    alphanum_key = lambda key: [convert(c) for c in re.split('([0-9]+)', key)]
    return sorted(l, key=alphanum_key)


for layer_path in natural_sort(os.listdir(nerf_layer_dir)):
    layer_path = os.path.join(nerf_layer_dir, layer_path)
    
    with open(layer_path, "r") as f:
        layer_config = yaml.safe_load(f)
        
    C = layer_config['problem']['instance']['C']
    M = layer_config['problem']['instance']['M']
    N = layer_config['problem']['instance']['N']
    print(f"{os.path.basename(layer_path)}, C={C}, M={M}, N={N}")

Nerf_layer1.yaml, C=63, M=256, N=16384
Nerf_layer2.yaml, C=256, M=256, N=16384
Nerf_layer3.yaml, C=256, M=256, N=16384
Nerf_layer4.yaml, C=256, M=256, N=16384
Nerf_layer5.yaml, C=256, M=256, N=16384
Nerf_layer6.yaml, C=319, M=256, N=16384
Nerf_layer7.yaml, C=256, M=256, N=16384
Nerf_layer8.yaml, C=256, M=256, N=16384
Nerf_layer9.yaml, C=256, M=1, N=16384
Nerf_layer10.yaml, C=256, M=256, N=16384
Nerf_layer11.yaml, C=283, M=128, N=16384
Nerf_layer12.yaml, C=128, M=3, N=16384


In [5]:
"VanillaNeRF Architecture", [model.modules]

('VanillaNeRF Architecture',
 [<bound method Module.modules of VanillaNeRF(
    (posi_encoder): SinusoidalEncoder()
    (view_encoder): SinusoidalEncoder()
    (mlp): NerfMLP(
      (base): MLP(
        (hidden_activation): ReLU()
        (output_activation): Identity()
        (hidden_layers): ModuleList(
          (0): Linear(in_features=63, out_features=256, bias=True)
          (1): Linear(in_features=256, out_features=256, bias=True)
          (2): Linear(in_features=256, out_features=256, bias=True)
          (3): Linear(in_features=256, out_features=256, bias=True)
          (4): Linear(in_features=256, out_features=256, bias=True)
          (5): Linear(in_features=319, out_features=256, bias=True)
          (6): Linear(in_features=256, out_features=256, bias=True)
          (7): Linear(in_features=256, out_features=256, bias=True)
        )
      )
      (sigma_layer): DenseLayer(
        (hidden_activation): ReLU()
        (output_activation): Identity()
        (hidden_layers